In [1]:
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

from bokeh.io import output_notebook, show
from bokeh.models import DataRange1d
from bokeh.layouts import gridplot
from bokeh.plotting import figure
output_notebook()

def plot_config(width: int, height: int,title=None,
                x_label: str = None, 
                y_label: str = None):
    p = figure(title=title, background_fill_color="#fafafa", width=width, height=height)
    p.output_backend = "svg"    # save plot in HTML as SVG
    p.title.text_font_size = "15pt"
    p.xaxis.axis_label = x_label
    p.yaxis.axis_label = y_label
    p.yaxis.axis_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"
    return p

Loading BokehJS ...

In [2]:
file_name = 'KF'

df_sim = pd.read_csv('./sim_result/' + file_name + '.csv')
df_sim.drop('Unnamed: 0', axis=1, inplace=True)

df_tf = df_sim[df_sim['sensor.type'] == 'tf']
df_ve = df_sim[df_sim['sensor.type'] == 'vel']

t0 = df_sim['time'][0]
tn = df_sim['time'].iloc[-1]

ti = np.array(df_tf['time'] - t0).reshape(-1,1)
xi = np.array(df_tf['tf.x']).reshape(-1)
yi = np.array(df_tf['tf.y']).reshape(-1)
thetai = np.array(df_tf['tf.yaw_z']).reshape(-1)
tvi = np.array(df_ve['time'] - t0).reshape(-1, 1)
vi = np.array(df_ve['velocity']).reshape(-1)


In [4]:

# condi1 = 5 <= df_sim['time'] -t0
# condi2 = df_sim['time'] -t0 <= 30
# df_window = df_sim[condi1*condi2]
# # df_window

# dx = df_window['x'].to_numpy()# - x_gpr
# dy = df_window['y'].to_numpy()# - y_gpr
# # np.sqrt(dx**2)
# SAE = np.sum(dx)
# print(SAE)
# print("{:.4f}".format(SAE))

In [3]:
kernel = 3 * RBF(length_scale=0.7, length_scale_bounds='fixed')

gpr_x = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0)
gpr_x.fit(ti, xi)
gpr_y = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0)
gpr_y.fit(ti, yi)
gpr_theta = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0)
gpr_theta.fit(ti, thetai)
gpr_v = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0)
gpr_v.fit(tvi, vi)

xp, stdx = gpr_x.predict(df_sim['time'].to_numpy().reshape(-1,1) - t0, return_std=True)
yp, stdy = gpr_y.predict(df_sim['time'].to_numpy().reshape(-1,1) - t0, return_std=True)
thetap, stdtheta = gpr_theta.predict(df_sim['time'].to_numpy().reshape(-1,1) - t0, return_std=True)
vp, stdv = gpr_v.predict(df_sim['time'].to_numpy().reshape(-1,1) - t0, return_std=True)

gpr_x.kernel_

1.73**2 * RBF(length_scale=0.7)

In [4]:
p_total = []
p01 = plot_config(width=600, height=400, title="state 'x'", x_label='t [ s ]', y_label='x [ m ] ')
p01.circle(x=df_sim['time'] - t0, y=xp, legend_label='gpr', size=2, fill_color='royalblue',line_color='royalblue')
p01.circle(x=df_sim['time'] - t0, y=df_sim['tf.x'], legend_label='sensor data', size=2, fill_color='black',line_color='black')
p_total.append(p01)

p02 = plot_config(width=600, height=400, title="state 'y'", x_label='t [ s ]', y_label='x [ m ] ')
p02.circle(x=df_sim['time'] - t0, y=yp, legend_label='gpr', size=2, fill_color='royalblue',line_color='royalblue')
p02.circle(x=df_sim['time'] - t0, y=df_sim['tf.y'], legend_label='sensor data', size=2, fill_color='black',line_color='black')
p_total.append(p02)

p03 = plot_config(width=600, height=400, title="state 'theta'", x_label='t [ s ]', y_label='x [ m ] ')
p03.circle(x=df_sim['time'] - t0, y=thetap, legend_label='gpr', size=2, fill_color='royalblue',line_color='royalblue')
p03.circle(x=df_sim['time'] - t0, y=df_sim['tf.yaw_z'], legend_label='sensor data', size=2, fill_color='black',line_color='black')
p_total.append(p03)

p04 = plot_config(width=600, height=400, title="state 'v'", x_label='t [ s ]', y_label='x [ m ] ')
p04.circle(x=df_sim['time'] - t0, y=vp, legend_label='gpr', size=2, fill_color='royalblue',line_color='royalblue')
p04.circle(x=df_sim['time'] - t0, y=df_sim['velocity'], legend_label='sensor data', size=2, fill_color='black',line_color='black')
p_total.append(p04)


for i in p_total:
    show(i, notebook_handle=True)